In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, 
    precision_score,
    recall_score,
    f1_score, 
    confusion_matrix,
    classification_report
)
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# 1. CHARGEMENT ET PRÉPARATION DES DONNÉES
# ============================================================================

# Charger les données
df = pd.read_csv("datas.csv")

print(f"Dataset chargé : {df.shape[0]} observations, {df.shape[1]} colonnes")
print(f"\nDistribution de la cible :")
print(df['koi_disposition'].value_counts())

# Sélection des colonnes pertinentes
selected_columns = [
    'koi_disposition',      # CIBLE (Confirmed / Candidate / False Positive)
    'koi_score',            # score de confiance du transit
    'koi_period',           # période orbitale [jours]
    'koi_duration',         # durée du transit [heures]
    'koi_depth',            # profondeur du transit [ppm]
    'koi_model_snr',        # signal-to-noise du transit
    'koi_prad',             # rayon planétaire [R_⊕]
    'koi_teq',              # température d'équilibre [K]
    'koi_fpflag_nt',        # transit non ressemblant (faux positif)
    'koi_fpflag_ss'         # éclipse stellaire (faux positif)
]

df_clean = df[selected_columns].copy()

# ============================================================================
# 2. NETTOYAGE DES DONNÉES
# ============================================================================

# Afficher les valeurs manquantes
print("\nValeurs manquantes par colonne :")
print(df_clean.isnull().sum())

# Remplir les valeurs manquantes numériques par la médiane
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col != 'koi_disposition':
        median_value = df_clean[col].median()
        df_clean[col].fillna(median_value, inplace=True)

# Supprimer les lignes où la cible est manquante
df_clean = df_clean.dropna(subset=['koi_disposition'])

print(f"\nDataset nettoyé : {df_clean.shape[0]} observations")

# ============================================================================
# 3. FEATURES ENGINEERING
# ============================================================================

# Créer des features supplémentaires basées sur les relations physiques

# Ratio rayon/période (indicateur de densité orbitale)
df_clean['radius_period_ratio'] = df_clean['koi_prad'] / np.sqrt(df_clean['koi_period'])

# Score de faux positif composite
df_clean['fp_flag_sum'] = df_clean['koi_fpflag_nt'] + df_clean['koi_fpflag_ss']

# Logarithme de certaines features pour normaliser leur distribution
df_clean['log_period'] = np.log1p(df_clean['koi_period'])
df_clean['log_depth'] = np.log1p(df_clean['koi_depth'])
df_clean['log_snr'] = np.log1p(df_clean['koi_model_snr'])

# ============================================================================
# 4. SÉPARATION FEATURES / TARGET
# ============================================================================

# Variable cible
target = 'koi_disposition'
y = df_clean[target]

# Features (exclure la cible)
X = df_clean.drop(columns=[target])

print(f"\nFeatures utilisées : {list(X.columns)}")

# ============================================================================
# 5. ENCODAGE DE LA CIBLE
# ============================================================================

# Encoder la variable cible (classification multi-classe)
label_encoder_target = LabelEncoder()
y_encoded = label_encoder_target.fit_transform(y)

print(f"\nClasses encodées :")
for i, label in enumerate(label_encoder_target.classes_):
    print(f"  {label} -> {i}")

# ============================================================================
# 6. SPLIT TRAIN/TEST
# ============================================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\nTrain set : {X_train.shape[0]} observations")
print(f"Test set  : {X_test.shape[0]} observations")

# ============================================================================
# 7. STANDARDISATION
# ============================================================================

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ============================================================================
# 8. ENTRAÎNEMENT DES MODÈLES
# ============================================================================

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='rbf', random_state=42)
}

results = {}

print("\n" + "="*70)
print("ENTRAÎNEMENT ET ÉVALUATION DES MODÈLES")
print("="*70)

for name, model in models.items():
    print(f"\n{'─'*70}")
    print(f"Modèle : {name}")
    print(f"{'─'*70}")
    
    # Entraînement
    model.fit(X_train_scaled, y_train)
    
    # Prédictions
    y_pred = model.predict(X_test_scaled)
    
    # Métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    results[name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "CV Mean": cv_mean,
        "CV Std": cv_std
    }
    
    print(f"Accuracy  : {accuracy:.4f}")
    print(f"Precision : {precision:.4f}")
    print(f"Recall    : {recall:.4f}")
    print(f"F1-Score  : {f1:.4f}")
    print(f"CV Score  : {cv_mean:.4f} (±{cv_std:.4f})")

# ============================================================================
# 9. HYPERPARAMETER TUNING (RANDOM FOREST)
# ============================================================================

print("\n" + "="*70)
print("HYPERPARAMETER TUNING - RANDOM FOREST")
print("="*70)

param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

# Meilleur modèle
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)

# Métriques du meilleur modèle
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best, average='weighted', zero_division=0)
recall_best = recall_score(y_test, y_pred_best, average='weighted', zero_division=0)
f1_best = f1_score(y_test, y_pred_best, average='weighted', zero_division=0)

print(f"\nMeilleurs hyperparamètres :")
print(grid_search.best_params_)
print(f"\nPerformances du modèle optimisé :")
print(f"Accuracy  : {accuracy_best:.4f}")
print(f"Precision : {precision_best:.4f}")
print(f"Recall    : {recall_best:.4f}")
print(f"F1-Score  : {f1_best:.4f}")

results["Random Forest Tuned"] = {
    "Accuracy": accuracy_best,
    "Precision": precision_best,
    "Recall": recall_best,
    "F1-Score": f1_best,
    "CV Mean": grid_search.best_score_,
    "CV Std": 0
}

# ============================================================================
# 10. RAPPORT DE CLASSIFICATION DÉTAILLÉ
# ============================================================================

print("\n" + "="*70)
print("RAPPORT DE CLASSIFICATION - MEILLEUR MODÈLE")
print("="*70)
print("\n" + classification_report(
    y_test, 
    y_pred_best, 
    target_names=label_encoder_target.classes_,
    zero_division=0
))

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred_best)
print("\nMatrice de confusion :")
print(cm)

# ============================================================================
# 11. IMPORTANCE DES FEATURES
# ============================================================================

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n" + "="*70)
print("IMPORTANCE DES FEATURES")
print("="*70)
print(feature_importance.to_string(index=False))

# ============================================================================
# 12. SAUVEGARDE DU MODÈLE
# ============================================================================

joblib.dump(best_model, "best_exoplanet_model.pkl")
joblib.dump(scaler, "exoplanet_scaler.pkl")
joblib.dump(label_encoder_target, "exoplanet_label_encoder.pkl")

print("\n" + "="*70)
print("MODÈLE SAUVEGARDÉ")
print("="*70)
print("Fichiers créés :")
print("  - best_exoplanet_model.pkl")
print("  - exoplanet_scaler.pkl")
print("  - exoplanet_label_encoder.pkl")

# ============================================================================
# 13. RÉSUMÉ COMPARATIF
# ============================================================================

results_df = pd.DataFrame(results).T
print("\n" + "="*70)
print("RÉSUMÉ COMPARATIF DES MODÈLES")
print("="*70)
print(results_df.to_string())

# Sauvegarder les résultats
results_df.to_csv("model_comparison_results.csv")
print("\nRésultats sauvegardés dans : model_comparison_results.csv")

# ============================================================================
# 14. EXEMPLE DE PRÉDICTION
# ============================================================================

print("\n" + "="*70)
print("EXEMPLE DE PRÉDICTION")
print("="*70)

# Charger le modèle sauvegardé
loaded_model = joblib.load("best_exoplanet_model.pkl")
loaded_scaler = joblib.load("exoplanet_scaler.pkl")
loaded_encoder = joblib.load("exoplanet_label_encoder.pkl")

# Exemple de données (première ligne du test set)
example = X_test.iloc[0:1]
example_scaled = loaded_scaler.transform(example)
prediction = loaded_model.predict(example_scaled)
prediction_proba = loaded_model.predict_proba(example_scaled)

print("\nDonnées d'entrée :")
print(example.to_string())
print(f"\nPrédiction : {loaded_encoder.inverse_transform(prediction)[0]}")
print("\nProbabilités par classe :")
for i, class_name in enumerate(loaded_encoder.classes_):
    print(f"  {class_name}: {prediction_proba[0][i]:.4f}")

print("\n" + "="*70)
print("ANALYSE TERMINÉE")
print("="*70)

Dataset chargé : 9564 observations, 49 colonnes

Distribution de la cible :
koi_disposition
FALSE POSITIVE    4839
CONFIRMED         2746
CANDIDATE         1979
Name: count, dtype: int64

Valeurs manquantes par colonne :
koi_disposition       0
koi_score          1510
koi_period            0
koi_duration          0
koi_depth           363
koi_model_snr       363
koi_prad            363
koi_teq             363
koi_fpflag_nt         0
koi_fpflag_ss         0
dtype: int64

Dataset nettoyé : 9564 observations

Features utilisées : ['koi_score', 'koi_period', 'koi_duration', 'koi_depth', 'koi_model_snr', 'koi_prad', 'koi_teq', 'koi_fpflag_nt', 'koi_fpflag_ss', 'radius_period_ratio', 'fp_flag_sum', 'log_period', 'log_depth', 'log_snr']

Classes encodées :
  CANDIDATE -> 0
  CONFIRMED -> 1
  FALSE POSITIVE -> 2

Train set : 7651 observations
Test set  : 1913 observations

ENTRAÎNEMENT ET ÉVALUATION DES MODÈLES

──────────────────────────────────────────────────────────────────────
Modèle : Lo

/tmp/ipykernel_530752/497619568.py:61: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(median_value, inplace=True)
/tmp/ipykernel_530752/497619568.py:61: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

Accuracy  : 0.8881
Precision : 0.8866
Recall    : 0.8881
F1-Score  : 0.8869
CV Score  : 0.8791 (±0.0017)

──────────────────────────────────────────────────────────────────────
Modèle : Decision Tree
──────────────────────────────────────────────────────────────────────
Accuracy  : 0.8589
Precision : 0.8604
Recall    : 0.8589
F1-Score  : 0.8594
CV Score  : 0.8581 (±0.0050)

──────────────────────────────────────────────────────────────────────
Modèle : Random Forest
──────────────────────────────────────────────────────────────────────
Accuracy  : 0.9085
Precision : 0.9088
Recall    : 0.9085
F1-Score  : 0.9086
CV Score  : 0.9000 (±0.0028)

──────────────────────────────────────────────────────────────────────
Modèle : Gradient Boosting
──────────────────────────────────────────────────────────────────────
Accuracy  : 0.9064
Precision : 0.9070
Recall    : 0.9064
F1-Score  : 0.9067
CV Score  : 0.9012 (±0.0041)

──────────────────────────────────────────────────────────────────────
Modèle